In [ ]:
!wget https://dl.fbaipublicfiles.com/MLQA/MLQA_V1.zip

In [ ]:
import zipfile
with zipfile.ZipFile('/kaggle/working/MLQA_V1.zip') as zip_ref:
    zip_ref.extractall('/kaggle/working/')

In [ ]:
import os
import sys
import random
import argparse
import json
import nltk
import numpy as np
import pandas as pd
from tqdm import tqdm

# sys.setdefaultencoding('utf8')
random.seed(42)
np.random.seed(42)

In [ ]:
mlqa_train_data = '/kaggle/working/MLQA_V1/dev/dev-context-hi-question-hi.json'
mlqa_test_data = '/kaggle/working/MLQA_V1/test/test-context-hi-question-hi.json'

with open(mlqa_train_data, 'r') as file_input:
    train_file = json.load(file_input)
    
with open(mlqa_test_data, 'r') as file_input:
    test_file = json.load(file_input)

In [ ]:
def preprocess(dataset, tier):
    num_exs = 0 
    examples = []

    for articles_id in tqdm(range(len(dataset['data'])), desc="Preprocessing {}".format(tier)):
        article_paragraphs = dataset['data'][articles_id]['paragraphs']
        for pid in range(len(article_paragraphs)):
            context = article_paragraphs[pid]['context']
            context = context.replace("''", '" ')
            context = context.replace("``", '" ')
            qas = article_paragraphs[pid]['qas'] 
            for qn in qas:
                question = qn['question'] 
                ans_text = qn['answers'][0]['text']
                ans_start_charloc = qn['answers'][0]['answer_start']
                ans_end_charloc = ans_start_charloc + len(ans_text)
                examples.append(
                    {
                        # 'id':articles_id,
                        'context':context, 
                        'question':question, 
                        'answer_text':ans_text, 
                        'answer_start':ans_start_charloc, 
                        # 'answer_end':ans_end_charloc
                    }
                )

                num_exs += 1
    print(num_exs)    
    return examples

In [ ]:
examples_train = preprocess(train_file, 'dev')
examples_test = preprocess(test_file, 'test')

In [ ]:
examples = examples_train + examples_test
mlqa = pd.DataFrame(examples)
mlqa['language'] = 'hindi'

In [ ]:
mlqa.head()

In [ ]:
!git clone https://github.com/deepmind/xquad.git

In [ ]:
xquad_train_file = '/kaggle/working/xquad/xquad.hi.json'

with open(xquad_train_file, 'r') as file_input:
    train_file = json.load(file_input)
    
examples_train = preprocess(train_file, 'dev')
xquad = pd.DataFrame(examples_train)
xquad['language'] = 'hindi'

In [ ]:
xquad.head()

In [ ]:
mlqa.head()

In [ ]:
mlqa.shape

In [ ]:
xquad.shape

In [ ]:
merged = pd.concat([mlqa, xquad])

In [ ]:
merged.shape

In [ ]:
merged.to_csv('merged_external.csv', index=False)

In [ ]:
!ls